In [1]:
#read datafolder from Google Drive
from google.colab import drive
drive.mount('/content/drive')
data_folder = '/content/drive/My Drive/reuters21578/'


sgml_number_of_files = 21
sgml_file_name_template = 'reut2-NNN.sgm'

# Category files
category_files = {
    'to_': ('Topics', 'all-topics-strings.lc.txt'),
    'pl_': ('Places', 'all-places-strings.lc.txt'),
    'pe_': ('People', 'all-people-strings.lc.txt'),
    'or_': ('Organizations', 'all-orgs-strings.lc.txt'),
    'ex_': ('Exchanges', 'all-exchanges-strings.lc.txt')
}

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [0]:
# Create category dataframe

# Read all categories
category_data = []
category_dictionary={'Topics':[],'Places':[],'People':[],'Organizations':[],'Exchanges':[]}
for category_prefix in category_files.keys():
    with open(data_folder + category_files[category_prefix][1], 'r') as file:
        for category in file.readlines():
            category_data.append([category_prefix + category.strip().lower(), 
                                  category_files[category_prefix][0]])

# Create category dataframe
for i in category_data:
#     print(i[1])
    category_dictionary[i[1]].append(i[0].split('_')[1])
news_categories = pd.DataFrame(data=category_data)

# print "category_data: ", category_data
#(news_categories.values).tolist()

In [4]:
import re
import xml.sax.saxutils as saxutils
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('stopwords')

Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
lemmatizer = WordNetLemmatizer()

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;\\n]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def cleanUpSentence(r):#, stop_words = None#
    r = r.lower().replace("<br />", " ")
    r = REPLACE_BY_SPACE_RE.sub(' ', r)
    r = BAD_SYMBOLS_RE.sub('', r)

    r = ' '.join(word for word in r.split() if word not in STOPWORDS)

    words = word_tokenize(r)

    for w in words:
        w = lemmatizer.lemmatize(w)

    return r

In [0]:
# Parse SGML files
def strip_tags(text):
    return re.sub('<[^<]+?>', '', text).strip()

def unescape(text):
    return saxutils.unescape(text)
  
def makeDict(filename, document_X):
  with open(filename, 'rb') as file:

    content = BeautifulSoup(file.read().lower(),'html.parser')

    for newsline in content('reuters'):
      document_categories = []

      document_id = newsline['newid']
      document_body = strip_tags(str(newsline('text')[0].body)).replace('reuter\n&#3;', '')
      if document_body == 'None':
        continue

      doc_categories = strip_tags(str(newsline('topics')[0].body))
      doc_categories = unescape(doc_categories)

      document_body = unescape(document_body)

      document_X[document_id] = document_body

def readFiles(test_data = False):
  document_X = {}
  
  if test_data == True:
    file_name = sgml_file_name_template.replace('NNN', '021')
    filename = data_folder + file_name
    makeDict(filename, document_X)
  else:
    for i in range(sgml_number_of_files):
      if i < 10:
        seq = '00' + str(i)
      else:
        seq = '0' + str(i)

      file_name = sgml_file_name_template.replace('NNN', seq)
      print('Reading file: %s' % file_name)
      filename = data_folder + file_name
      makeDict(filename, document_X)
  return document_X



In [7]:
document_X= readFiles()

Reading file: reut2-000.sgm
Reading file: reut2-001.sgm
Reading file: reut2-002.sgm
Reading file: reut2-003.sgm
Reading file: reut2-004.sgm
Reading file: reut2-005.sgm
Reading file: reut2-006.sgm
Reading file: reut2-007.sgm
Reading file: reut2-008.sgm
Reading file: reut2-009.sgm
Reading file: reut2-010.sgm
Reading file: reut2-011.sgm
Reading file: reut2-012.sgm
Reading file: reut2-013.sgm
Reading file: reut2-014.sgm
Reading file: reut2-015.sgm
Reading file: reut2-016.sgm
Reading file: reut2-017.sgm
Reading file: reut2-018.sgm
Reading file: reut2-019.sgm
Reading file: reut2-020.sgm


In [8]:
# data preprocessing
import numpy as np

nltk.download('punkt')
nltk.download('wordnet')

def create_x_matrix(document_X):
    totalX = []
    for i, doc in document_X.items():
        totalX.append(cleanUpSentence(doc))
    max_vocab_size = 200
    input_tokenizer = Tokenizer(200)
    input_tokenizer.fit_on_texts(totalX)
    encoded_docs = input_tokenizer.texts_to_matrix(totalX, mode='count')
    return totalX,encoded_docs

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
totalX,encoded_docs=create_x_matrix(document_X)

In [0]:
#Create one-hot encode
def makeOneHotEncoding(totalX):
  words_in_body={}

  for i in range(len(totalX)):
      words=totalX[i].split(' ')
      words_in_body[i]=words    

  one_hot_label=[]
  for key,v in words_in_body.items():
      dict_temp={'Topics':0,'Places':0,'People':0,'Exchanges':0,'Organizations':0}
      for i in v:
          if i in category_dictionary['Topics']:
              dict_temp['Topics']+=1
          if i in category_dictionary['Places']:
              dict_temp['Places']+=1
          if i in category_dictionary['People']:
              dict_temp['People']+=1
          if i in category_dictionary['Exchanges']:
              dict_temp['Exchanges']+=1
          if i in category_dictionary['Organizations']:
              dict_temp['Organizations']+=1

      one_hot_label.append(dict_temp)


  one_hot_label_list = []
  for i in one_hot_label:

      one_hot_label_list.append(list(i.values()))
  return one_hot_label_list

In [0]:
one_hot_label_list = makeOneHotEncoding(totalX)

In [23]:
from keras.models import Sequential
from keras.layers import Dense,Flatten, Dropout,Embedding
nn = Sequential()
max_vocab_size = 200
nn.add(Embedding(200, 20, input_length=max_vocab_size))
nn.add(Dense(10, activation="relu", input_shape=(max_vocab_size,)))
nn.add(Dropout(0.15))
nn.add(Flatten())
nn.add(Dense(5,activation="sigmoid"))
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
nn.fit(np.array(encoded_docs), np.array(one_hot_label_list), batch_size=16, epochs=5,
          verbose=1, validation_split=0.2)

Train on 14866 samples, validate on 3717 samples
Epoch 1/5
14866/14866 [==============================] - 4s 247us/step - loss: -1.8747 - acc: 0.6365 - val_loss: -2.6152 - val_acc: 0.6738
Epoch 2/5
14866/14866 [==============================] - 3s 231us/step - loss: -2.7531 - acc: 0.6749 - val_loss: -2.8500 - val_acc: 0.6891
Epoch 3/5
14866/14866 [==============================] - 3s 230us/step - loss: -2.9317 - acc: 0.6871 - val_loss: -2.9464 - val_acc: 0.6788
Epoch 4/5
14866/14866 [==============================] - 3s 228us/step - loss: -2.9972 - acc: 0.6923 - val_loss: -2.9411 - val_acc: 0.6893
Epoch 5/5
14866/14866 [==============================] - 3s 231us/step - loss: -3.0353 - acc: 0.6958 - val_loss: -2.9764 - val_acc: 0.6789


In [24]:
test_document_X = readFiles(test_data=True)
print(len(test_document_X))

460


In [25]:
test_total_X,test_encoded_X= create_x_matrix(test_document_X)
ground_truth_list = makeOneHotEncoding(test_total_X)
y = nn.predict(test_encoded_X)
print (y)

[[1.0000000e+00 1.0000000e+00 9.9801874e-01 1.8446982e-02 2.2590160e-05]
 [1.0000000e+00 1.0000000e+00 5.6062198e-01 4.0322244e-03 9.8329449e-01]
 [1.0000000e+00 8.3306360e-01 1.0407865e-03 9.9986434e-01 4.5827329e-03]
 ...
 [1.0000000e+00 6.0533476e-01 1.6038030e-02 5.7339966e-03 4.1027963e-03]
 [1.0000000e+00 2.6603383e-01 9.9999958e-01 8.7005794e-03 2.6977062e-04]
 [1.0000000e+00 1.0000000e+00 9.9999499e-01 5.2462395e-02 1.7900596e-03]]


In [0]:
def permissible(x, y):
  if (abs(x-y)) < 2:
    return True
  else:
    return False

def accuracy(predicted,actual):
    tp=0
    fp = 0
    tp_list = []
    length = len(actual)
    print("length: ",length)
    for one_doc_idx in range(length):
        if permissible(predicted[one_doc_idx][0],actual[one_doc_idx][0]) and permissible(predicted[one_doc_idx][1],actual[one_doc_idx][1])\
          and permissible(predicted[one_doc_idx][2],actual[one_doc_idx][2]) and permissible(predicted[one_doc_idx][3],actual[one_doc_idx][3]):
            tp+=1
            x = tp
        else:
          fp = 0
          x = fp
        tp_list.append(x)
    return tp/float(length), tp_list

In [27]:
def getRankedOutput2(predicted_y):
    all_sorted_x = []
    for i in predicted_y:
        dict_temp={'Topics':i[0],'Places':i[1],'Peoples':i[2],'Exchanges':i[3],'Organizations':i[4]}
        sorted_x = sorted(dict_temp.items(), key=lambda kv: kv[1], reverse=True) 
        all_sorted_x.append(sorted_x)
    return all_sorted_x
   
acc, tpl = accuracy(y.tolist(), ground_truth_list)
print ("%f"%acc)
predicted_ranked_output = getRankedOutput2(y)
actual_ranked_output=getRankedOutput2(np.array(ground_truth_list))

length:  460
0.836957


In [28]:
my_df  = pd.DataFrame(columns = ['body', 'predicted ranking', 'actual ranking', '_'])
my_df['body'] = test_total_X
my_df['predicted ranking'] = predicted_ranked_output 
my_df['actual ranking'] = actual_ranked_output
my_df['_'] = tpl
my_df = my_df.sort_values(by='_', ascending=False).reset_index(drop=True)
my_df[['body', 'predicted ranking', 'actual ranking']]

,body,predicted ranking,actual ranking
0,prospect dominant alliance socialists environm...,"[(Topics, 1.0), (Peoples, 0.9999996), (Places,...","[(Places, 1), (Topics, 0), (Peoples, 0), (Exch..."
1,japan indiapakistangulf japan shipping confere...,"[(Topics, 1.0), (Places, 0.98393154), (Peoples...","[(Places, 2), (Topics, 0), (Peoples, 0), (Exch..."
2,announcement alliant computer systems inc alnt...,"[(Topics, 1.0), (Places, 1.0), (Peoples, 1.0),...","[(Exchanges, 1), (Topics, 0), (Places, 0), (Pe..."
3,chase corp ltd chcawe said make offer fullypai...,"[(Topics, 1.0), (Peoples, 0.9955495), (Places,...","[(Topics, 1), (Places, 0), (Peoples, 0), (Exch..."
4,thai rice exports rose 72 987 tonnes week ende...,"[(Topics, 1.0), (Places, 0.06996614), (Organiz...","[(Topics, 2), (Places, 1), (Peoples, 0), (Exch..."
5,orders nonfuel imports placed august rose seas...,"[(Topics, 1.0), (Places, 0.22204566), (Exchang...","[(Topics, 2), (Places, 0), (Peoples, 0), (Exch..."
6,new zealand imposed sanctions fiji response co...,"[(Topics, 1.0), (Places, 0.9999628), (Peoples,...","[(Topics, 2), (Places, 2), (Organizations, 2),..."
7,iranian president ali khamenei said doubted us...,"[(Topics, 1.0), (Places, 0.99999803), (Peoples...","[(Places, 2), (Peoples, 1), (Organizations, 1)..."
8,philippines trade deficit widened 542 mln dlrs...,"[(Topics, 1.0), (Peoples, 0.9999873), (Places,...","[(Topics, 2), (Places, 1), (Peoples, 0), (Exch..."
9,nippon shinpan co ltd nshtt agreed mastercard ...,"[(Topics, 1.0), (Places, 0.9118398), (Organiza...","[(Places, 1), (Topics, 0), (Peoples, 0), (Exch..."
